![oi](../images/header.png)

## Preprocessamento de dados
---
**Professores:**
- Dr. Gilberto Queiroz
- Dr. Rafael Santos

**Discente:**
- Adriano Almeida

<a id="indice"></a>

## Índice
---
- [1. Importação de módulos e bibliotecas](#importacao-de-modulos-e-bibliotecas)
- [2. Conjunto de dados](#conjunto-de-dados)
    - [2.1 Obtendo diretório dos arquivos](#obtendo-diretorio-dos-arquivos)
    - [2.2 Carregando conjunto de dados](#carregando-conjunto-de-dados)
    - [2.3 Gravando arquivos em CSV](#gravando-arquivos-em-csv)
- [3. Referências](#referencias)

[🏠 Início](https://github.com/AdrianoPereira/project-cap394/)

<a id="importacao-de-modulos-e-bibliotecas"></a>
## 1. Importação de módulos e bibliotecas [^](#indice)
---

Para realizar o preprocessamento de dados serão feitas as seguintes importações:
- **os** - Este é um módulo que já vem com a instalação padrão do Python. Ele fornece funcionalidades de manipulação de arquivos e diretórios de forma idependente do sistema operacional.

-  **numpy** - Esta é uma biblioteca que possibilita ler, criar e manipular manipular estruturas como vetores e matrizes de múltiplas dimensões. Além disso, a biblioteca `Numpy` possui uma coleção de funções matemáticas que possibilitam trabalhar com essas estruturas.  


- **pandas** - É uma biblioteca para leitura, manipulação e análise de dados tabulados. Essa biblioteca oferece estruturas de dados e operações para manipular conjuntos massivos de tabelas numéricas e séries temporais de forma otimizada.

Por convensão, as bibliotecas são importadas no python da seguinte maneira:

In [1]:
import os
import numpy as np
import pandas as pd

<a id="conjunto-de-dados"></a>

## 2. Conjunto de dados [^](#indice)

---

Os dados são oriundos dos experimentos CHUVA-Manaus <a href="#machado-et-al-2014">(MACHADO et al., 2014)</a> e GoAmazon <a href="#martin-et-al-2016">(MARTIN et al., 2016)</a>, e estão estão agrupados por tabelas em arquivos no formato `.txt`. Esses arquivos representam a saída do processamento realizado por Rebeca e Rachel <a href="#rebeca-2019">(REBECA, 2019)</a> utilizando o argoritmo **Forecast and Tracking the Evolution of Cloud Clusters - ForTraCC** <a href="#vila-et-al-2008">(VILA et al., 2008)</a>. Os dados possuem resolução temporal de 12 minutos, e resolução espacial de 1 km, e foram coletados nos meses de fevereiro, março, agosto, setembro e outubro de 2014.

Existem dois tipos de arquivos:
- **fam_allequip_SEQN_tttt_FAMN_TTTT_IDATE_YYYYMMDDHHmm.txt**


    - Este tipo de arquivo representa as famílias (agrupamentos de nuvens) em cada tempo. Este arquivos também apresenta um resumos dos índices de cada família.


- **Matrix_fam_allequip_SEQN_tttt_FAMN_TTTT_IDATE_YYYYMMDDHHmm.txt**


    - Este tipo de arquivo exibe toda a área de cobertura, que é representada por uma grade com dimensões de 241x241. 
    
<div class="alert alert-block alert-info">
    <b>Info:</b> as substrings `tttt` e `TTTT` no nome do arquivo representam o intervalo em que foram capturados os dados. As substrings `YYYY`, `MM`, `DD`, `HH` e `mm` representam respectivamente o ano, dia, hora e minuto em que foram capturados os dados.
</div>

<a id="obtendo-diretorio-dos-arquivos"></a>
### 2.1 Obtendo diretório dos arquivos [^](#indice)

***

Ambos os tipos de dados estão localizados no seguinte diretório: `/data/private/output_fortracc`. Exitem ao todo 43750 arquivos, sendo 21875 de cada tipo de arquivo, como mostrado abaixo.

In [2]:
#setting file paths
PATH_PRIVATE_DATA = "../data/private/output_fortracc" 

#summarizing data from type fam
!echo "Total de arquivos do tipo fam:" && \
ls {PATH_PRIVATE_DATA}|grep ^fam|wc -l && \
echo "Primeiro 5 arquivos do tipo fam:" && \
ls {PATH_PRIVATE_DATA} | grep ^fam | head -5

#summarizing data from type Matrix
!echo "\nTotal de arquivos do tipo Matrix:" && \
ls {PATH_PRIVATE_DATA}|grep ^Matrix|wc -l && \
echo "Primeiro 5 arquivos do tipo Matrix:" && \
ls {PATH_PRIVATE_DATA} | grep ^Matrix | head -5

Total de arquivos do tipo fam:
21875
Primeiro 5 arquivos do tipo fam:
fam_allequip_SEQN_0000_FAMN_0001_IDATE_201402010000.txt
fam_allequip_SEQN_0000_FAMN_0001_IDATE_201403010000.txt
fam_allequip_SEQN_0000_FAMN_0001_IDATE_201408010000.txt
fam_allequip_SEQN_0000_FAMN_0001_IDATE_201409011612.txt
fam_allequip_SEQN_0000_FAMN_0001_IDATE_201410010000.txt
grep: write error: Broken pipe
ls: write error: Broken pipe

Total de arquivos do tipo Matrix:
21875
Primeiro 5 arquivos do tipo Matrix:
Matrix_fam_allequip_SEQN_0000_FAMN_0001_IDATE_201402010000.txt
Matrix_fam_allequip_SEQN_0000_FAMN_0001_IDATE_201403010000.txt
Matrix_fam_allequip_SEQN_0000_FAMN_0001_IDATE_201408010000.txt
Matrix_fam_allequip_SEQN_0000_FAMN_0001_IDATE_201409011612.txt
Matrix_fam_allequip_SEQN_0000_FAMN_0001_IDATE_201410010000.txt
grep: write error: Broken pipe
ls: write error: Broken pipe


Afim de facilitar a leitura dos dados e torná-los *tidy*, eles serão convertidos para o formato `.csv`, separados em pastas diferentes e agrupados por mês. Primeiramente, será necessário definir o diretório dos arquivos e carregá-los como um dataframe.

In [3]:
#listing and storing filenames of each type
PATH_FILES_FAM = !ls {PATH_PRIVATE_DATA}|grep ^fam
PATH_FILES_MAT = !ls {PATH_PRIVATE_DATA}|grep ^Matrix

#adding full path of each file
PATH_FILES_FAM = [os.path.join(PATH_PRIVATE_DATA, filename)\
                 for filename in PATH_FILES_FAM]
PATH_FILES_MAT = [os.path.join(PATH_PRIVATE_DATA, filename)\
                 for filename in PATH_FILES_MAT]
PATH_FILES_FAM[:5]

['../data/private/output_fortracc/fam_allequip_SEQN_0000_FAMN_0001_IDATE_201402010000.txt',
 '../data/private/output_fortracc/fam_allequip_SEQN_0000_FAMN_0001_IDATE_201403010000.txt',
 '../data/private/output_fortracc/fam_allequip_SEQN_0000_FAMN_0001_IDATE_201408010000.txt',
 '../data/private/output_fortracc/fam_allequip_SEQN_0000_FAMN_0001_IDATE_201409011612.txt',
 '../data/private/output_fortracc/fam_allequip_SEQN_0000_FAMN_0001_IDATE_201410010000.txt']

<a id="carregando-conjunto-de-dados"></a>
### 2.2 Carregando conjunto de dados [^](#indice)

---

Como a forma de carregamento dos arquivos é igual para os dois tipos de dados, será definida uma função para isso. Essa função deverá receber a lista de arquivos que foi processada anteriormente.

In [4]:
#defining function to load files as dataframe
def load_files_as_dataframe(filenames, add_time=False):
    #function axiliar to load dataframe
    def _load_data(filename):
        df = pd.read_csv(filename, sep='\s+')
        
        #adding time in dataframe
        if add_time:
            df['year'] = int(filename[-16:-12])
            df['month'] = int(filename[-12:-10])
            df['day'] = int(filename[-10:-8])
            df['hour'] = int(filename[-8:-6])
            df['minute'] = int(filename[-6:-4])
        
        return df
    
    #loading files as dataframe and storing in list
    dfs = [_load_data(filename) \
              for filename in filenames]
        

    #joining dataframes into single dataframe
    dfs = pd.concat(dfs, sort=False)

    #turning header to lowercase
    dfs = dfs.rename(str.lower, axis='columns')
    
    return dfs

famdfs = load_files_as_dataframe(PATH_FILES_FAM, add_time=True)
famdfs.head()

,time,sysclass,lat,lon,dir,vel,size,ttyyyxx1,ttyyyxx2,ttyyyxx3,...,maxz,meanvil,ttvil,meanprec,maxprec,year,month,day,hour,minute
0,0.0,0.0,-1.73,-60.57,-999.0,0.0,53.0,0.0,0.0,0.0,...,26.70,0.29,15.37,1.18,3.18,2014,2,1,0,0
1,0.2,1.0,-1.73,-60.55,90.0,11.1,21.0,0.0,0.0,0.0,...,21.46,0.23,4.91,0.92,1.03,2014,2,1,0,0
2,0.4,1.0,-1.77,-60.53,153.0,24.8,18.0,0.0,0.0,0.0,...,24.83,0.24,4.24,1.15,2.35,2014,2,1,0,0
0,0.0,0.0,-1.99,-60.93,-999.0,0.0,180.0,0.0,0.0,0.0,...,28.80,0.41,73.93,1.60,4.52,2014,3,1,0,0
1,0.2,1.0,-2.07,-60.87,143.0,55.5,123.0,0.0,0.0,0.0,...,31.51,0.46,56.63,2.17,6.97,2014,3,1,0,0


In [13]:
matdfs = load_files_as_dataframe(PATH_FILES_MAT)
matdfs.head()

,year,month,day,hour,minute,n_cluster,lon,lat,ind_x,ind_y,reflect,yyyyy_xx1,yyyyy_xx2,yyyyyyy_xx3,yyy_xx4,yyy_xx5,river,cxe,precipit,vil
0,2014.0,2.0,1.0,0.0,0.0,3.0,-60.6314,-1.6689,88.0,46.0,20.2031,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.8128,0.1902
1,2014.0,2.0,1.0,0.0,0.0,3.0,-60.5514,-1.6689,92.0,46.0,20.7031,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.8776,0.3290
2,2014.0,2.0,1.0,0.0,0.0,3.0,-60.5314,-1.6689,93.0,46.0,20.7031,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.8776,0.3288
3,2014.0,2.0,1.0,0.0,0.0,3.0,-60.6714,-1.6889,86.0,47.0,21.2031,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.9477,0.1914
4,2014.0,2.0,1.0,0.0,0.0,3.0,-60.6514,-1.6889,87.0,47.0,21.2031,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.9477,0.1922


<a id="gravando-arquivos-em-csv"></a>

### 2.3 Gravando arquivos em CSV [^](#indice)
Uma vez carregado todos os arquivos, eles deverão ser salvos no formato `.csv` de forma separada, como mencionado anteriormente.

In [14]:
def save_data_as_csv(dataframe, directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    #saving in single dataframe every month
    dataframe.to_csv('%s/all_months.csv'%directory, sep=',', \
                     index=False)
    
    months = [
        (2, 'february'), (3, 'march'), (8, 'august'), 
        (9, 'september'), (10, 'october')
    ]
    
    for month in months:
        df = dataframe[dataframe['month'] == month[0]]
        if df.shape[0]:
            df.to_csv('%s/%s.csv'%(directory, month[1]), \
                      sep=',', index=False)
        
    
# save_data_as_csv(famdfs, '../data/private/csv/fam')
save_data_as_csv(matdfs, '../data/private/csv/mat')

<a id="referencias"></a>

## 3. Referências [^](#indice)

<a id="machado-et-al-2014"></a>

MACHADO, L. A. T.; SILVA DIAS, M. A. F.; MORALES, C.; FISCH, G.; VILA, D.; ALBRECHT, R. I.; GOODMAN, S. J.; CALHEIROS, A. J. P.; BISCARO, T.; KUMMEROW, C.; COHEN, J.; FITZJARRALD, D.; NASCIMENTO, E. L.; SAKAMOTO, M. S.; CUNNINGHAM, C.; CHABOUREAU, J.-P.; PETERSEN, W. a.; ADAMS, D. K.; BALDINI, L.; ANGELIS, C. F.; SAPUCCI, L. F.; SALIO, P.; BARBOSA, H. M. J.; LANDULFO, E.; SOUZA, R. a. F.; BLAKESLEE, R. J.; BAILEY, J.; FREITAS, S.; LIMA, W. F. a.; TOKAY, A. The Chuva Project: How Does Convection Vary across Brazil? Bulletin of the American Meteorological Society, v. 95, n. 9, p. 1365–1380, set. 2014.

<a id="martin-et-al-2016"></a>

MARTIN, S. T.; ARTAXO, P.; MACHADO, L. A. T.; MANZI, A. O.; SOUZA, R. A. F.; SCHUMACHER, C.; WANG, J.; ANDREAE, M. O.; BARBOSA, H. M. J.; FAN, J.; FISCH, G.; GOLDSTEIN, A. H.; GUENTHER, A.; JIMENEZ, J. L.; PÖSCHL, U.; SILVA DIAS, M. A.; 99 SMITH, J. N.; WENDISCH, M. Introduction: Observations and Modeling of the Green Ocean Amazon (GoAmazon2014/5). Atmospheric Chemistry and Physics, v. 16, n. 8, p. 4785–4797, abr. 2016.

<a id="rebeca-2019"></a>

PEREIRA, R. F. d. O. (2019). Propriedades físicas da precipitação e eletrificação dos sistemas precipitantes observados durante o projeto chuva-manaus e goamazon. Dissertação de mestrado, Universidade de São Paulo (USP).

<a id="vila-et-al-2008"></a>

VILA, D. A., MACHADO, L. A. T., LAURENT, H., & VELASCO, I. (2008). Forecast and Tracking the Evolution of Cloud Clusters (ForTraCC) using satellite infrared imagery: Methodology and validation. Weather and Forecasting, 23(2), 233-245.